In [ ]:
import pandas as pd

DF = pd.read_excel('one_off_conversation_sad.xlsx', engine='openpyxl') # 슬픔만 분류해둔 엑셀 파일 불러오기

DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5267 entries, 0 to 5266
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  5267 non-null   object
 1   emotion   5267 non-null   object
dtypes: object(2)
memory usage: 82.4+ KB


In [ ]:
DF_answer = pd.read_excel('one_off_conversation_sad_answer.xlsx', engine='openpyxl')
# 사람인 맞춤법 검사기 + 네이버 맞춤법 검사기 돌린 정답 파일 불러오기 (100문장만 처리했음)

DF_answer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5267 entries, 0 to 5266
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  5267 non-null   object
 1   emotion   5267 non-null   object
dtypes: object(2)
memory usage: 82.4+ KB


In [ ]:
sentence = pd.DataFrame() # 빈 데이터프레임 생성

In [ ]:
sentence['original'] = DF['sentence'] # 불러온 엑셀에서 문장만 sentence 데이터프레임의 original 열에 추가

sentence.head()

,original
0,어깨라도넓엇으면 옷빨이라도 잘살텐데..
1,그저 보고싶어죽겟어요
2,아 진짜 저한테는 악운만 붙는거같네요ㅠㅠ........0
3,지금 계단에서울고있어요
4,저때문에 택배하나 배달하려다 변을 당하신건 아니겠죠0


In [ ]:
remove_empty = []

for s in sentence['original']:
    remove_empty.append(s.strip())

In [ ]:
sentence['original'] = pd.DataFrame(remove_empty)

In [ ]:
sentence.head()

,original
0,어깨라도넓엇으면 옷빨이라도 잘살텐데..
1,그저 보고싶어죽겟어요
2,아 진짜 저한테는 악운만 붙는거같네요ㅠㅠ........0
3,지금 계단에서울고있어요
4,저때문에 택배하나 배달하려다 변을 당하신건 아니겠죠0


In [ ]:
remove_empty = []

for s in DF_answer['sentence']:
    remove_empty.append(s.strip())

In [ ]:
DF_answer['sentence'] = pd.DataFrame(remove_empty)

In [ ]:
%%time

from hanspell import spell_checker

hanspell_result = []

for s in sentence['original']:
    hanspell = spell_checker.check(s) # hanspell 활용하여 띄어쓰기 및 맞춤법 검사
    hanspell_result.append(hanspell.checked)

CPU times: user 12.7 s, sys: 215 ms, total: 12.9 s
Wall time: 3min 19s


In [ ]:
sentence['hanspell'] = pd.DataFrame(hanspell_result) # hanspell 결과 sentence 데이터프레임에 저장

In [ ]:
%%time

from pykospacing import spacing

spacing_sentence = []

for s in sentence['hanspell']:
    kospacing_sentence = spacing(s) # pykospacing 활용하여 띄어쓰기
    spacing_sentence.append(kospacing_sentence)

CPU times: user 3min 35s, sys: 8.96 s, total: 3min 44s
Wall time: 3min 33s


In [ ]:
sentence['pykospacing'] = pd.DataFrame(spacing_sentence) # pykospacing 결과 sentence 데이터프레임에 저장

In [ ]:
answer_100 = DF_answer.iloc[:100].copy() # 100문장만 처리했기 때문에 100개만 자르기

answer_100.tail()

,sentence,emotion
95,혼자서 먹고 있다...,슬픔
96,이제 이 세상 살기 힘들어지는구나,슬픔
97,미래가 없네....,슬픔
98,먹고살기도 바쁜데 결혼은 할 수 있을까?,슬픔
99,생각만 해도 벌써 가슴이 먹먹하다,슬픔


In [ ]:
sentence_100 = sentence.iloc[:100].copy() # 정답과 비교 위해 전처리한 데이터프레임도 100개만 자름

sentence_100.tail()

,original,hanspell,pykospacing
95,혼자서 먹고있다...,혼자서 먹고 있다...,혼자서 먹고 있다...
96,이제 이세상 살기 힘들어지는구나,이제 이 세상 살기 힘들어지는구나,이제 이 세상 살기 힘들어지는구나
97,미래가 없네....,미래가 없네....,미래가 없네....
98,먹고 살기도 바쁜데 결혼은 할수 있을까?,먹고살기도 바쁜데 결혼은 할 수 있을까?,먹고 살기도 바쁜데 결혼은 할 수 있을까?
99,생각만 해도 벌써 가슴이 먹먹하다,생각만 해도 벌써 가슴이 먹먹하다,생각만 해도 벌써 가슴이 먹먹하다


In [ ]:
sentence_100['hanspell_only_korean'] = sentence_100['hanspell'].str.replace('[^가-힣]', '') 

In [ ]:
sentence_100['hanspell_with_num'] = sentence_100['hanspell'].str.replace('[^가-힣0-9]', '') 

In [ ]:
sentence_100['hanspell_with_alphabet'] = sentence_100['hanspell'].str.replace('[^가-힣0-9A-Za-z]', '') 

In [ ]:
answer_100['sentence_only_korean'] = answer_100['sentence'].str.replace('[^가-힣]', '')

In [ ]:
answer_100['sentence_with_num'] = answer_100['sentence'].str.replace('[^가-힣0-9]', '')

In [ ]:
answer_100['sentence_with_alphabet'] = answer_100['sentence'].str.replace('[^가-힣0-9A-Za-z]', '')

In [ ]:
sentence_100['hanspell_with_everything'] = sentence_100['hanspell'].str.replace('[^가-힣0-9A-Za-zㄱ-ㅎㅏ-ㅣ]', '')

In [ ]:
answer_100['sentence_with_everything'] = answer_100['sentence'].str.replace('[^가-힣0-9A-Za-zㄱ-ㅎㅏ-ㅣ]', '')

In [ ]:
sentence_100

,original,hanspell,pykospacing,hanspell_only_korean,hanspell_with_num,hanspell_with_alphabet,hanspell_with_everything
0,어깨라도넓엇으면 옷빨이라도 잘살텐데..,어깨라도 넓었으면 옷발이라도 잘 살 텐데..,어깨라도 넓었으면 옷 발이라도 잘 살 텐데..,어깨라도넓었으면옷발이라도잘살텐데,어깨라도넓었으면옷발이라도잘살텐데,어깨라도넓었으면옷발이라도잘살텐데,어깨라도넓었으면옷발이라도잘살텐데
1,그저 보고싶어죽겟어요,그저 보고 싶어 죽겠어요,그저 보고 싶어 죽겠어요,그저보고싶어죽겠어요,그저보고싶어죽겠어요,그저보고싶어죽겠어요,그저보고싶어죽겠어요
2,아 진짜 저한테는 악운만 붙는거같네요ㅠㅠ........0,아 진짜 저한테는 악운만 붙는거같네요ㅠㅠ........0,아 진짜 저한테는 악운만 붙는 거 같네요 ㅠㅠ........0,아진짜저한테는악운만붙는거같네요,아진짜저한테는악운만붙는거같네요0,아진짜저한테는악운만붙는거같네요0,아진짜저한테는악운만붙는거같네요ㅠㅠ0
3,지금 계단에서울고있어요,지금 계단에서 울고 있어요,지금 계단에서 울고 있어요,지금계단에서울고있어요,지금계단에서울고있어요,지금계단에서울고있어요,지금계단에서울고있어요
4,저때문에 택배하나 배달하려다 변을 당하신건 아니겠죠0,저 때문에 택배 하나 배달하려다 변을 당하신 건 아니겠죠 0,저 때문에 택배 하나 배달하려다 변을 당하신 건 아니겠죠 0,저때문에택배하나배달하려다변을당하신건아니겠죠,저때문에택배하나배달하려다변을당하신건아니겠죠0,저때문에택배하나배달하려다변을당하신건아니겠죠0,저때문에택배하나배달하려다변을당하신건아니겠죠0
...,...,...,...,...,...,...,...
95,혼자서 먹고있다...,혼자서 먹고 있다...,혼자서 먹고 있다...,혼자서먹고있다,혼자서먹고있다,혼자서먹고있다,혼자서먹고있다
96,이제 이세상 살기 힘들어지는구나,이제 이 세상 살기 힘들어지는구나,이제 이 세상 살기 힘들어지는구나,이제이세상살기힘들어지는구나,이제이세상살기힘들어지는구나,이제이세상살기힘들어지는구나,이제이세상살기힘들어지는구나
97,미래가 없네....,미래가 없네....,미래가 없네....,미래가없네,미래가없네,미래가없네,미래가없네
98,먹고 살기도 바쁜데 결혼은 할수 있을까?,먹고살기도 바쁜데 결혼은 할 수 있을까?,먹고 살기도 바쁜데 결혼은 할 수 있을까?,먹고살기도바쁜데결혼은할수있을까,먹고살기도바쁜데결혼은할수있을까,먹고살기도바쁜데결혼은할수있을까,먹고살기도바쁜데결혼은할수있을까


In [ ]:
answer_100

,sentence,emotion,sentence_only_korean,sentence_with_num,sentence_with_alphabet,sentence_with_everything
0,어깨라도 넓었으면 옷발이라도 잘 살 텐데..,슬픔,어깨라도넓었으면옷발이라도잘살텐데,어깨라도넓었으면옷발이라도잘살텐데,어깨라도넓었으면옷발이라도잘살텐데,어깨라도넓었으면옷발이라도잘살텐데
1,그저 보고 싶어 죽겠어요,슬픔,그저보고싶어죽겠어요,그저보고싶어죽겠어요,그저보고싶어죽겠어요,그저보고싶어죽겠어요
2,아 진짜 저한테는 악운만 붙는 거 같네요. ㅠㅠ........,슬픔,아진짜저한테는악운만붙는거같네요,아진짜저한테는악운만붙는거같네요,아진짜저한테는악운만붙는거같네요,아진짜저한테는악운만붙는거같네요ㅠㅠ
3,지금 계단에서 울고 있어요,슬픔,지금계단에서울고있어요,지금계단에서울고있어요,지금계단에서울고있어요,지금계단에서울고있어요
4,저 때문에 택배 하나 배달하려다 변을 당하신 건 아니겠죠,슬픔,저때문에택배하나배달하려다변을당하신건아니겠죠,저때문에택배하나배달하려다변을당하신건아니겠죠,저때문에택배하나배달하려다변을당하신건아니겠죠,저때문에택배하나배달하려다변을당하신건아니겠죠
...,...,...,...,...,...,...
95,혼자서 먹고 있다...,슬픔,혼자서먹고있다,혼자서먹고있다,혼자서먹고있다,혼자서먹고있다
96,이제 이 세상 살기 힘들어지는구나,슬픔,이제이세상살기힘들어지는구나,이제이세상살기힘들어지는구나,이제이세상살기힘들어지는구나,이제이세상살기힘들어지는구나
97,미래가 없네....,슬픔,미래가없네,미래가없네,미래가없네,미래가없네
98,먹고살기도 바쁜데 결혼은 할 수 있을까?,슬픔,먹고살기도바쁜데결혼은할수있을까,먹고살기도바쁜데결혼은할수있을까,먹고살기도바쁜데결혼은할수있을까,먹고살기도바쁜데결혼은할수있을까


In [ ]:
correct_original = 0

for i in range(100):
    if sentence_100['hanspell'][i] == answer_100['sentence'][i]: 
        correct_original += 1
    else:
        print(f"{sentence_100['hanspell'][i]} <-> {answer_100['sentence'][i]}")

아 진짜 저한테는 악운만 붙는거같네요ㅠㅠ........0 <-> 아 진짜 저한테는 악운만 붙는 거 같네요. ㅠㅠ........
저 때문에 택배 하나 배달하려다 변을 당하신 건 아니겠죠 0 <-> 저 때문에 택배 하나 배달하려다 변을 당하신 건 아니겠죠
잊은 줄 알았는데 잊을 수 있다고 생각했는데 갑자기 쏟아지는 눈물에  강남 한복판에서 주저앉게 되더라.. <-> 잊은 줄 알았는데 잊을 수 있다고 생각했는데 갑자기 쏟아지는 눈물에 강남 한복판에서 주저앉게 되더라..
근데 같이 갈 사람이 없다는 게 안타깝네요.. ㅜㅜ <-> 근데 같이 갈 사람이 없다는 게 안타깝네요..ㅜㅜ
유일하게 남은 게 그분 내의나 냉장고에 붙혀진 시 한 편이 고작입니다. <-> 유일하게 남은 게 그분 내의나 냉장고에 붙여진 시 한 편이 고작입니다.
초등학생인데요 좋아하는 아이를 그만 접으려고요 <-> 초딩인데요 좋아하는 아이를 그만 접으려고요
남자친구가 이번 달 말에 입대하는데 ㅠㅠ <-> 남친이 이번 달 말에 입대하는데 ㅠㅠ
모쏠인데 어떻게 하면 연애할수있냐ㅠㅠ <-> 모쏠인데 어떻게 하면 연애 할 수 있냐ㅠㅠ
그 눈빛을 난 잊을 수가 없어 오래됐는데... <-> 그 눈빛을 난 잊을 수가 없어 오래 됐는데...
제 여자친구는 절 좋아하는 거 같지 않고... <-> 제 여친은 절 좋아하는 거 같지 않고...
혼자 떨어져 있는 느낌이 들어서 활발하게 대하고 놀고 싶은데 그게 안돼요ᅲ <-> 혼자 떨어져 있는 느낌이 들어서 활발하게 대하고 놀고 싶은데 그게 안돼요ㅠ
저는 아직까지 한 통의 전화 와 편지가 오지 않았네요ㅠ <-> 저는 아직까지 한 통의 전화와 편지가 오지 않았네요ㅠ
흑 남자친구도 없고 회사는 너무 바쁘고.. <-> 흑 남친도 없고 회사는 너무 바쁘고..
부상과 열악한 연습 환경, 온 국민의 기대의 부담감을 안고 얼마나 힘들었을까요... <-> 부상과 열악한 연습 환경, 온 국민의 기대 부담감을 안고 얼마나 힘들었을까요...
어휴 아직도 화요일이라니..ㅠㅠ <-> 에휴 아직도 화요

In [ ]:
correct_korean = 0

for i in range(100):
    if sentence_100['hanspell_only_korean'][i] == answer_100['sentence_only_korean'][i]: 
        correct_korean += 1
    else:
        print(f"{sentence_100['hanspell_only_korean'][i]} <-> {answer_100['sentence_only_korean'][i]}")

유일하게남은게그분내의나냉장고에붙혀진시한편이고작입니다 <-> 유일하게남은게그분내의나냉장고에붙여진시한편이고작입니다
초등학생인데요좋아하는아이를그만접으려고요 <-> 초딩인데요좋아하는아이를그만접으려고요
남자친구가이번달말에입대하는데 <-> 남친이이번달말에입대하는데
제여자친구는절좋아하는거같지않고 <-> 제여친은절좋아하는거같지않고
흑남자친구도없고회사는너무바쁘고 <-> 흑남친도없고회사는너무바쁘고
부상과열악한연습환경온국민의기대의부담감을안고얼마나힘들었을까요 <-> 부상과열악한연습환경온국민의기대부담감을안고얼마나힘들었을까요
어휴아직도화요일이라니 <-> 에휴아직도화요일이라니
하루에도몇번씩추억에심장이쿵하고내려앉고어느특정한날짜만나와도아이때는내가너랑헤어지기전이였네후였네오나뉜다 <-> 하루에도몇번씩추억에심장이쿵하고내려앉고어느특정한날짜만나와도아이때는내가너랑헤어지기전이였네후였네로나뉜다
전전남자친구와헤어졌을때정말죽을만큼힘들었어요 <-> 전전남친과헤어졌을때정말죽을만큼힘들었어요
요즘몸도아프고돈나갈때도많아지고안그래도힘든데날씨까지추워지니지치네요 <-> 요즘몸도아프고돈나갈데도많아지고안그래도힘든데날씨까지추워지니지치네요
친구들에게남자친구아이갖고싶다니까미쳤대요 <-> 친구들에게남친아이갖고싶다니까미쳤대요
미래를본다면제가훨씬나을수도있지만현재로썬정말힘드네요 <-> 미래를본다면제가훨씬나을수도있지만현재로선정말힘드네요
이여자아니면다시는여자친구못만날거같습니다 <-> 이여자아니면다시는여자친구못만날것같습니다


In [ ]:
correct_num = 0

for i in range(100):
    if sentence_100['hanspell_with_num'][i] == answer_100['sentence_with_num'][i]: 
        correct_num += 1
    else:
        print(f"{sentence_100['hanspell_with_num'][i]} <-> {answer_100['sentence_with_num'][i]}")

아진짜저한테는악운만붙는거같네요0 <-> 아진짜저한테는악운만붙는거같네요
저때문에택배하나배달하려다변을당하신건아니겠죠0 <-> 저때문에택배하나배달하려다변을당하신건아니겠죠
유일하게남은게그분내의나냉장고에붙혀진시한편이고작입니다 <-> 유일하게남은게그분내의나냉장고에붙여진시한편이고작입니다
초등학생인데요좋아하는아이를그만접으려고요 <-> 초딩인데요좋아하는아이를그만접으려고요
남자친구가이번달말에입대하는데 <-> 남친이이번달말에입대하는데
제여자친구는절좋아하는거같지않고 <-> 제여친은절좋아하는거같지않고
흑남자친구도없고회사는너무바쁘고 <-> 흑남친도없고회사는너무바쁘고
부상과열악한연습환경온국민의기대의부담감을안고얼마나힘들었을까요 <-> 부상과열악한연습환경온국민의기대부담감을안고얼마나힘들었을까요
어휴아직도화요일이라니 <-> 에휴아직도화요일이라니
하루에도몇번씩추억에심장이쿵하고내려앉고어느특정한날짜만나와도아이때는내가너랑헤어지기전이였네후였네오나뉜다 <-> 하루에도몇번씩추억에심장이쿵하고내려앉고어느특정한날짜만나와도아이때는내가너랑헤어지기전이였네후였네로나뉜다
전전남자친구와헤어졌을때정말죽을만큼힘들었어요 <-> 전전남친과헤어졌을때정말죽을만큼힘들었어요
요즘몸도아프고돈나갈때도많아지고안그래도힘든데날씨까지추워지니지치네요 <-> 요즘몸도아프고돈나갈데도많아지고안그래도힘든데날씨까지추워지니지치네요
친구들에게남자친구아이갖고싶다니까미쳤대요 <-> 친구들에게남친아이갖고싶다니까미쳤대요
미래를본다면제가훨씬나을수도있지만현재로썬정말힘드네요 <-> 미래를본다면제가훨씬나을수도있지만현재로선정말힘드네요
이여자아니면다시는여자친구못만날거같습니다 <-> 이여자아니면다시는여자친구못만날것같습니다


In [ ]:
correct_alphabet = 0

for i in range(100):
    if sentence_100['hanspell_with_alphabet'][i] == answer_100['sentence_with_alphabet'][i]: 
        correct_alphabet += 1
    else:
        print(f"{sentence_100['hanspell_with_alphabet'][i]} <-> {answer_100['sentence_with_alphabet'][i]}")

아진짜저한테는악운만붙는거같네요0 <-> 아진짜저한테는악운만붙는거같네요
저때문에택배하나배달하려다변을당하신건아니겠죠0 <-> 저때문에택배하나배달하려다변을당하신건아니겠죠
유일하게남은게그분내의나냉장고에붙혀진시한편이고작입니다 <-> 유일하게남은게그분내의나냉장고에붙여진시한편이고작입니다
초등학생인데요좋아하는아이를그만접으려고요 <-> 초딩인데요좋아하는아이를그만접으려고요
남자친구가이번달말에입대하는데 <-> 남친이이번달말에입대하는데
제여자친구는절좋아하는거같지않고 <-> 제여친은절좋아하는거같지않고
흑남자친구도없고회사는너무바쁘고 <-> 흑남친도없고회사는너무바쁘고
부상과열악한연습환경온국민의기대의부담감을안고얼마나힘들었을까요 <-> 부상과열악한연습환경온국민의기대부담감을안고얼마나힘들었을까요
어휴아직도화요일이라니 <-> 에휴아직도화요일이라니
하루에도몇번씩추억에심장이쿵하고내려앉고어느특정한날짜만나와도아이때는내가너랑헤어지기전이였네후였네오나뉜다 <-> 하루에도몇번씩추억에심장이쿵하고내려앉고어느특정한날짜만나와도아이때는내가너랑헤어지기전이였네후였네로나뉜다
전전남자친구와헤어졌을때정말죽을만큼힘들었어요 <-> 전전남친과헤어졌을때정말죽을만큼힘들었어요
요즘몸도아프고돈나갈때도많아지고안그래도힘든데날씨까지추워지니지치네요 <-> 요즘몸도아프고돈나갈데도많아지고안그래도힘든데날씨까지추워지니지치네요
친구들에게남자친구아이갖고싶다니까미쳤대요 <-> 친구들에게남친아이갖고싶다니까미쳤대요
미래를본다면제가훨씬나을수도있지만현재로썬정말힘드네요 <-> 미래를본다면제가훨씬나을수도있지만현재로선정말힘드네요
이여자아니면다시는여자친구못만날거같습니다 <-> 이여자아니면다시는여자친구못만날것같습니다


In [ ]:
correct_kospacing = 0

for i in range(100):
    if sentence_100['pykospacing'][i] == answer_100['sentence'][i]: 
        correct_kospacing += 1
    else:
        print(f"{sentence_100['pykospacing'][i]}\n{answer_100['sentence'][i]}")
        print()

어깨라도 넓었으면 옷 발이라도 잘 살 텐데..
어깨라도 넓었으면 옷발이라도 잘 살 텐데..

아 진짜 저한테는 악운만 붙는 거 같네요 ㅠㅠ........0
아 진짜 저한테는 악운만 붙는 거 같네요. ㅠㅠ........

저 때문에 택배 하나 배달하려다 변을 당하신 건 아니겠죠 0
저 때문에 택배 하나 배달하려다 변을 당하신 건 아니겠죠

근데 같이 갈 사람이 없다는 게 안타깝네요.. ㅜㅜ
근데 같이 갈 사람이 없다는 게 안타깝네요..ㅜㅜ

유일하게 남은 게 그분 내의 나 냉장고에 붙혀진 시 한 편이 고작입니다.
유일하게 남은 게 그분 내의나 냉장고에 붙여진 시 한 편이 고작입니다.

초등학생인데요 좋아하는 아이를 그만 접으려고요
초딩인데요 좋아하는 아이를 그만 접으려고요

남자친구가 이번 달 말에 입대하는데 ㅠㅠ
남친이 이번 달 말에 입대하는데 ㅠㅠ

콧방울이 너무 못 생겼어 ㅜㅜ
콧방울이 너무 못생겼어 ㅜㅜ

모쏠인데 어떻게 하면 연애할 수 있냐 ㅠㅠ
모쏠인데 어떻게 하면 연애 할 수 있냐ㅠㅠ

그 눈빛을 난 잊을 수가 없어 오래됐는데...
그 눈빛을 난 잊을 수가 없어 오래 됐는데...

제 여자친구는 절 좋아하는 거 같지 않고...
제 여친은 절 좋아하는 거 같지 않고...

혼자 떨어져 있는 느낌이 들어서 활발하게 대하고 놀고 싶은데 그게 안 돼요ᅲ
혼자 떨어져 있는 느낌이 들어서 활발하게 대하고 놀고 싶은데 그게 안돼요ㅠ

저는 아직까지 한 통의 전화 와 편지가 오지 않았네요ㅠ
저는 아직까지 한 통의 전화와 편지가 오지 않았네요ㅠ

흑 남자친구도 없고 회사는 너무 바쁘고..
흑 남친도 없고 회사는 너무 바쁘고..

부상과 열악한 연습 환경, 온 국민의 기대의 부담감을 안고 얼마나 힘들었을까요...
부상과 열악한 연습 환경, 온 국민의 기대 부담감을 안고 얼마나 힘들었을까요...

어휴 아직도 화요일이라니..ㅠㅠ
에휴 아직도 화요일이라니..ㅠㅠ

야구의 전설 중의 전설 최동원 님께서 암 투병 중에 별세 하심에 애도를 표합니다.
야구의 전설 중의 전설 최

In [ ]:
correct_everything = 0

for i in range(100):
    if sentence_100['hanspell_with_everything'][i] == answer_100['sentence_with_everything'][i]: 
        correct_everything += 1
    else:
        print(f"{sentence_100['hanspell_with_everything'][i]}\n{answer_100['sentence_with_everything'][i]}")
        print()

아진짜저한테는악운만붙는거같네요ㅠㅠ0
아진짜저한테는악운만붙는거같네요ㅠㅠ

저때문에택배하나배달하려다변을당하신건아니겠죠0
저때문에택배하나배달하려다변을당하신건아니겠죠

유일하게남은게그분내의나냉장고에붙혀진시한편이고작입니다
유일하게남은게그분내의나냉장고에붙여진시한편이고작입니다

초등학생인데요좋아하는아이를그만접으려고요
초딩인데요좋아하는아이를그만접으려고요

남자친구가이번달말에입대하는데ㅠㅠ
남친이이번달말에입대하는데ㅠㅠ

제여자친구는절좋아하는거같지않고
제여친은절좋아하는거같지않고

혼자떨어져있는느낌이들어서활발하게대하고놀고싶은데그게안돼요
혼자떨어져있는느낌이들어서활발하게대하고놀고싶은데그게안돼요ㅠ

흑남자친구도없고회사는너무바쁘고
흑남친도없고회사는너무바쁘고

부상과열악한연습환경온국민의기대의부담감을안고얼마나힘들었을까요
부상과열악한연습환경온국민의기대부담감을안고얼마나힘들었을까요

어휴아직도화요일이라니ㅠㅠ
에휴아직도화요일이라니ㅠㅠ

곰신분들저는어쩌면좋을까요
곰신분들저는어쩌면좋을까요ㅜㅜ

하루에도몇번씩추억에심장이쿵하고내려앉고어느특정한날짜만나와도아이때는내가너랑헤어지기전이였네후였네오나뉜다
하루에도몇번씩추억에심장이쿵하고내려앉고어느특정한날짜만나와도아이때는내가너랑헤어지기전이였네후였네로나뉜다

전전남자친구와헤어졌을때정말죽을만큼힘들었어요
전전남친과헤어졌을때정말죽을만큼힘들었어요

요즘몸도아프고돈나갈때도많아지고안그래도힘든데날씨까지추워지니지치네요
요즘몸도아프고돈나갈데도많아지고안그래도힘든데날씨까지추워지니지치네요

친구들에게남자친구아이갖고싶다니까미쳤대요
친구들에게남친아이갖고싶다니까미쳤대요

미래를본다면제가훨씬나을수도있지만현재로썬정말힘드네요
미래를본다면제가훨씬나을수도있지만현재로선정말힘드네요

이여자아니면다시는여자친구못만날거같습니다
이여자아니면다시는여자친구못만날것같습니다



In [ ]:
print(f'{correct_original / 100}')
print(f'{correct_korean / 100}')
print(f'{correct_num / 100}')
print(f'{correct_alphabet / 100}')
print(f'{correct_kospacing / 100}')
print(f'{correct_everything / 100}')

0.74
0.87
0.85
0.85
0.56
0.83
